<a href="https://colab.research.google.com/github/Degananda264/Movie-sentimental-analysis/blob/master/Moview_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tf-nightly
import tensorflow_datasets as tfds
import tensorflow as tf

     |████████████████████████████████| 522.4MB 32kB/s 
     |████████████████████████████████| 460kB 51.1MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 3.0MB 47.6MB/s 
  Found existing installation: Keras-Preprocessing 1.1.0
    Uninstalling Keras-Preprocessing-1.1.0:
      Successfully uninstalled Keras-Preprocessing-1.1.0


In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [4]:
  dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                            as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete0A891V/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete0A891V/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete0A891V/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [0]:
 train_examples, test_examples = dataset['train'], dataset['test']

In [6]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Pot

In [0]:
encoder = info.features['text'].encoder

In [14]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [0]:
train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE, padded_shapes=([None],[])))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE,  padded_shapes=([None],[])))

In [0]:
train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE))

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [0]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [0]:
checkpoint_path = "/content/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [80]:
history = model.fit(train_dataset, epochs=4,
                    validation_data=test_dataset,
                    validation_steps=30,callbacks=[cp_callback])

Epoch 1/4
391/391 [==============================] - ETA: 0s - loss: 0.0808 - accuracy: 0.9807
Epoch 00001: saving model to /content/training_1/cp.ckpt
391/391 [==============================] - 103s 264ms/step - loss: 0.0808 - accuracy: 0.9807 - val_loss: 0.5738 - val_accuracy: 0.8474
Epoch 2/4
391/391 [==============================] - ETA: 0s - loss: 0.0701 - accuracy: 0.9842
Epoch 00002: saving model to /content/training_1/cp.ckpt
391/391 [==============================] - 102s 261ms/step - loss: 0.0701 - accuracy: 0.9842 - val_loss: 0.5867 - val_accuracy: 0.8292
Epoch 3/4
391/391 [==============================] - ETA: 0s - loss: 0.0677 - accuracy: 0.9850
Epoch 00003: saving model to /content/training_1/cp.ckpt
391/391 [==============================] - 102s 260ms/step - loss: 0.0677 - accuracy: 0.9850 - val_loss: 0.6310 - val_accuracy: 0.8432
Epoch 4/4
391/391 [==============================] - ETA: 0s - loss: 0.0729 - accuracy: 0.9828
Epoch 00004: saving model to /content/traini

In [82]:
ls {checkpoint_dir}

checkpoint                   cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [84]:
model.load_weights(checkpoint_path)

In [85]:

test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 39s 100ms/step - loss: 0.6250 - accuracy: 0.8434
Test Loss: 0.6250074505805969
Test Accuracy: 0.8434000015258789


In [86]:
model.save('/content/saved_model/my_model')

INFO:tensorflow:Assets written to: /content/saved_model/my_model/assets


INFO:tensorflow:Assets written to: /content/saved_model/my_model/assets


In [0]:
new_model = tf.keras.models.load_model('/content/saved_model/my_model')

In [0]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [0]:
def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = new_model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [90]:
# predict on a sample text without padding.

sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

[[0.31942877]]


In [91]:
# predict on a sample text with padding

sample_pred_text = ('The movie was very excellent!!! The animation and the graphics '
                    'were oustanding. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

[[4.9215574]]
